# Backprojection image formation

In [ ]:
import torch
import torchbp
import matplotlib.pyplot as plt
from numpy import hamming

Use CUDA if it's available, CPU otherwise

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print("Device:", device)

Constant definitions

In [ ]:
nr = 128 # Range points
ntheta = 128 # Azimuth points
nsweeps = 128 # Number of measurements
fc = 6e9 # RF center frequency
bw = 100e6 # RF bandwidth
tsweep = 100e-6 # Sweep length
fs = 1e6 # Sampling frequency
nsamples = int(fs * tsweep) # Time domain samples per sweep

# Imaging grid definition. Azimuth angle "theta" is sine of radians. 0.2 = 11.5 degrees.
grid_polar = {"r": (90, 110), "theta": (-0.2, 0.2), "nr": nr, "ntheta": ntheta}

Define target and radar positions. There is one point target at 100 m distance and zero azimuth angle.
For polar image formation radar motion should be in direction of Y-axis.
If this is not the case positions should be rotated.

In [ ]:
target_pos = torch.tensor([[100, 0, 0]], dtype=torch.float32, device=device)
target_rcs = torch.tensor([[1]], dtype=torch.float32, device=device)
pos = torch.zeros([nsweeps, 3], dtype=torch.float32, device=device)
pos[:,1] = torch.linspace(-nsweeps/2, nsweeps/2, nsweeps) * 0.25 * 3e8 / fc

# Not used in this case
vel = torch.zeros((nsweeps, 3), dtype=torch.float32, device=device)
att = torch.zeros((nsweeps, 3), dtype=torch.float32, device=device)

Generate synthetic radar data

In [ ]:
# Oversampling input data decreases interpolation errors
oversample = 3

with torch.no_grad():
    data = torchbp.util.generate_fmcw_data(target_pos, target_rcs, pos, fc, bw, tsweep, fs)
    # Apply windowing function in range direction
    w = torch.tensor(hamming(data.shape[-1])[None,:], dtype=torch.float32, device=device)
    data = torch.fft.fft(data * w, dim=-1, n=nsamples * oversample)

data_db = 20*torch.log10(torch.abs(data)).detach()
m = torch.max(data_db)

plt.figure()
plt.imshow(data_db.cpu().numpy(), origin="lower", vmin=m-30, vmax=m, aspect="auto")
plt.xlabel("Range samples")
plt.ylabel("Azimuth samples");

Image formation.
Hamming window was applied in range direction so low sidelobes in range are expected.
Azimuth direction has no windowing function and high sidelobes (Highest -13 dB) are expected.
Azimuth sidelobes could be decreased by windowing the input data also in the other dimension.

In [ ]:
r_res = 3e8 / (2 * bw * oversample) # Range bin size in input data

img = torchbp.ops.backprojection_polar_2d(data, grid_polar, fc, r_res, pos, vel, att)
img = img.squeeze() # Removes singular batch dimension

img_db = 20*torch.log10(torch.abs(img)).detach()

m = torch.max(img_db)

extent = [*grid_polar["r"], *grid_polar["theta"]]

plt.figure()
plt.imshow(img_db.cpu().numpy().T, origin="lower", vmin=m-30, vmax=m, extent=extent, aspect="auto")
plt.xlabel("Range (m)")
plt.ylabel("Angle (sin radians)");

Image entropy. Can be used as a loss function for optimization.

In [ ]:
entropy = torchbp.util.entropy(img)
print("Entropy:", entropy.item())

Convert image to cartesian coordinates:

In [ ]:
# Origin of the polar coordinates
origin = torch.zeros(2, dtype=torch.float32, device=device)
# Cartesian grid definition
grid_cart = {"x": (90, 110), "y": (-10, 10), "nx": 128, "ny": 128}

# Complex image can also be interpolated, but takings absolute value first is faster
img_cart = torchbp.ops.polar_to_cart_linear(torch.abs(img), origin, grid_polar, grid_cart, fc, rotation=0)

img_db = 20*torch.log10(img_cart).detach()

m = torch.max(img_db)

extent = [*grid_cart["x"], *grid_cart["y"]]

plt.figure()
plt.imshow(img_db.cpu().numpy().T, origin="lower", vmin=m-30, vmax=m, extent=extent, aspect="equal")
plt.xlabel("Range (m)")
plt.ylabel("Cross-range (m)");